# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [18]:
# Load environment variables from .env file
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [26]:
import os
from glob import glob

# Load PRICE_DATA environment variable
price_data_dir = os.getenv("PRICE_DATA")

# Find all parquet files path in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive = True)

# Display the files
parquet_files


['../../05_src/data/prices\\AAN\\AAN_1982\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1982\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1983\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1983\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1984\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1984\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1985\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1985\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1986\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1986\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1987\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1987\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1988\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1988\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1989\\part.0.parquet',
 '../../05_src/data/prices\\AAN\\AAN_1989\\part.1.parquet',
 '../../05_src/data/prices\\AAN\\AAN_199

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [72]:

# Load parquet files into a Dask DataFrame
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px

# Bring ticker back as a column and sort within partitions (meta is specified to avoid warning)
dd_px = dd_px.reset_index()


# Sort within partitions with explicit meta
dd_px = dd_px.map_partitions(
    lambda pdf: pdf.sort_values(["ticker", "Date"]),
    meta=dd_px._meta
)

# Set ticker back as index (to keep original structure)
dd_px = dd_px.set_index("ticker")

# add lagged close and adjusted close columns
dd_px["Close_lag_1"] = dd_px.groupby("ticker")["Close"].shift(1)
dd_px["Adj_Close_lag_1"] = dd_px.groupby("ticker")["Adj Close"].shift(1)

# add returns and hi_lo_range columns

dd_px["returns"] = (dd_px["Close"] / dd_px["Close_lag_1"]) - 1
dd_px["hi_lo_range"] = dd_px["High"] - dd_px["Low"]

# Assign result
dd_feat = dd_px

dd_feat

C:\Users\h_daw\AppData\Local\Temp\ipykernel_28572\4229118263.py:19: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  dd_px["Close_lag_1"] = dd_px.groupby("ticker")["Close"].shift(1)
C:\Users\h_daw\AppData\Local\Temp\ipykernel_28572\4229118263.py:20: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  dd_px["Adj_Close_lag_1"] = dd_px.groupby("ticker")["Adj Close"].shift(1)


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=89,,,,,,,,,,,,,
AAN,datetime64[ns],float64,float64,float64,float64,float64,int64,string,int32,float64,float64,float64,float64
ABMD,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YYY,...,...,...,...,...,...,...,...,...,...,...,...,...
YYY,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [69]:
# Convert Dask DataFrame to Pandas
pdf = dd_feat.compute()

# Ensure correct ordering before rolling
pdf = pdf.sort_values(["ticker", "Date"])

# 10-day moving average of returns per ticker
pdf["returns_movingaverage_10"] = (
    pdf.groupby("ticker")["returns"]
       .rolling(10)
       .mean()
       .reset_index(level=0, drop=True)
)


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

You can compute the moving average directly in Dask. Converting to Pandas is often done for simplicity or small datasets, but Dask supports rolling operations as well.

yes, it would have been better to do it in Dask as the dataset is quite large.

Also as we were already using Dask for reading and feature creation, staying in Dask means we can keep everything lazy/distributed until the end.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.